In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras

In [2]:
df=pd.read_csv("healthcare-dataset-stroke-data.csv")

In [3]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df.drop('id',axis=1,inplace=True)

In [5]:
df.dtypes

gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [6]:
for column in df.columns:
    if df[column].dtype=='object':
     print(column,df[column].unique())

gender ['Male' 'Female' 'Other']
ever_married ['Yes' 'No']
work_type ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
Residence_type ['Urban' 'Rural']
smoking_status ['formerly smoked' 'never smoked' 'smokes' 'Unknown']


In [7]:
df['ever_married'].replace({'Yes':1,'No':0},inplace=True)

df['Residence_type'].replace({'Urban':1,'Rural':0},inplace=True)

In [8]:
df=pd.get_dummies(data=df,columns=['gender','work_type','smoking_status'])

In [9]:
df=df.dropna()

In [10]:
cols_to_scale = ['age','avg_glucose_level','bmi']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [11]:
df.dtypes

age                               float64
hypertension                        int64
heart_disease                       int64
ever_married                        int64
Residence_type                      int64
avg_glucose_level                 float64
bmi                               float64
stroke                              int64
gender_Female                       uint8
gender_Male                         uint8
gender_Other                        uint8
work_type_Govt_job                  uint8
work_type_Never_worked              uint8
work_type_Private                   uint8
work_type_Self-employed             uint8
work_type_children                  uint8
smoking_status_Unknown              uint8
smoking_status_formerly smoked      uint8
smoking_status_never smoked         uint8
smoking_status_smokes               uint8
dtype: object

In [12]:
df.head()

,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0.816895,0,1,1,1,0.801265,0.301260,1,0,1,0,0,0,1,0,0,0,1,0,0
2,0.975586,0,1,1,0,0.234512,0.254296,1,0,1,0,0,0,1,0,0,0,0,1,0
3,0.597168,0,0,1,1,0.536008,0.276060,1,1,0,0,0,0,1,0,0,0,0,0,1
4,0.963379,1,0,1,0,0.549349,0.156930,1,1,0,0,0,0,0,1,0,0,0,1,0
5,0.987793,0,0,1,1,0.605161,0.214204,1,0,1,0,0,0,1,0,0,0,1,0,0


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop('stroke',axis=1),df['stroke'],test_size=0.2)

In [14]:
y_train.value_counts()

0    3765
1     162
Name: stroke, dtype: int64

In [15]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy='minority')

x_train,y_train=smote.fit_resample(x_train,y_train)
x_test,y_test=smote.fit_resample(x_test,y_test)

In [16]:
y_train.value_counts()

0    3765
1    3765
Name: stroke, dtype: int64

In [29]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train,y_train, random_state=42)
x_test, y_test = shuffle(x_test,y_test, random_state=42)


In [18]:
print(len(x_train.columns))


19


In [30]:
model=keras.Sequential([
 keras.layers.Dense(32,input_shape=(19,),activation='relu'),
 keras.layers.Dropout(0.5),
 keras.layers.Dense(16,activation='relu'),
  keras.layers.Dropout(0.1),
 keras.layers.Dense(1,activation='sigmoid')
])

In [31]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [32]:
model.fit(x_train,y_train,epochs=500)

Epoch 1/500
236/236 [==============================] - 1s 4ms/step - loss: 0.6434 - accuracy: 0.6256
Epoch 2/500
236/236 [==============================] - 1s 4ms/step - loss: 0.5578 - accuracy: 0.7194
Epoch 3/500
236/236 [==============================] - 1s 4ms/step - loss: 0.5024 - accuracy: 0.7538
Epoch 4/500
236/236 [==============================] - 1s 3ms/step - loss: 0.4602 - accuracy: 0.7895
Epoch 5/500
236/236 [==============================] - 1s 4ms/step - loss: 0.4324 - accuracy: 0.8052
Epoch 6/500
236/236 [==============================] - 1s 3ms/step - loss: 0.4198 - accuracy: 0.8155
Epoch 7/500
236/236 [==============================] - 1s 3ms/step - loss: 0.4126 - accuracy: 0.8157
Epoch 8/500
236/236 [==============================] - 1s 4ms/step - loss: 0.4042 - accuracy: 0.8247
Epoch 9/500
236/236 [==============================] - 1s 4ms/step - loss: 0.3912 - accuracy: 0.8275
Epoch 10/500
236/236 [==============================] - 1s 4ms/step - loss: 0.3877 - accura

In [33]:
model.evaluate(x_test,y_test)

59/59 [==============================] - 0s 3ms/step - loss: 0.5845 - accuracy: 0.8128


[0.5844544172286987, 0.8128342032432556]

In [34]:
model.save('Stroke_prediction.h5')
